### Step 5: Model Deployment

In [10]:
suppressMessages(library('dplyr'))
suppressMessages(library('quantmod'))
suppressMessages(library('repr'))
suppressMessages(library('xlsx'))
suppressMessages(library('derivmkts'))
suppressMessages(library('ggplot2'))

#### Read in Data



In [20]:
TSLA_OPTIONS = xlsx::read.xlsx('Data/TSLA_OPTIONS.xlsx', sheetName = "Sheet1")

TSLA_STOCK = xlsx::read.xlsx('Data/TSLA_STOCK.xlsx', sheetName = "Sheet1")

In [21]:
n_row = dim(TSLA_OPTIONS)[1]
n_col = dim(TSLA_OPTIONS)[2]

print(paste(c("Total Number of Rows = ", n_row), collapse = ''))
print(paste(c("Total Number of Columns = ", n_col), collapse = ''))      

(TSLA_OPTIONS[1:5,])

[1] "Total Number of Rows = 1925"
[1] "Total Number of Columns = 16"


,Strike,Last,Chg,Bid,Ask,Vol,OI,LastTradeTime,IV,ITM,INDEX,Exp_date,TYPE,Snapshot_Date,Years_to_Exp_Date,Avg_Price
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,50,1145.55,16.600098,1162.7,1164.85,9,5,2021-11-03 19:20:48,18.63672,TRUE,TSLA211105C00050000,11/5/2021,CALL,11/3/2021,0.007936508,1163.78
2,100,1095.55,28.210083,1112.7,1114.85,7,7,2021-11-03 19:20:48,14.25781,TRUE,TSLA211105C00100000,11/5/2021,CALL,11/3/2021,0.007936508,1113.78
3,150,1044.30,6.060059,1062.7,1064.85,2,2,2021-11-03 19:00:58,11.85157,TRUE,TSLA211105C00150000,11/5/2021,CALL,11/3/2021,0.007936508,1063.78
4,200,995.55,5.599976,1012.7,1014.85,4,22,2021-11-03 19:20:48,10.20118,TRUE,TSLA211105C00200000,11/5/2021,CALL,11/3/2021,0.007936508,1013.78
5,250,927.30,-8.940002,962.7,964.85,1,1,2021-11-03 17:01:08,8.94727,TRUE,TSLA211105C00250000,11/5/2021,CALL,11/3/2021,0.007936508,963.78


In [23]:
n_row = dim(TSLA_STOCK)[1]
n_col = dim(TSLA_STOCK)[2]

print(paste(c("Total Number of Rows = ", n_row), collapse = ''))
print(paste(c("Total Number of Columns = ", n_col), collapse = '')) 


TSLA_STOCK$Date = as.character(TSLA_STOCK$Date)


[1] "Total Number of Rows = 717"
[1] "Total Number of Columns = 7"



The purpose of this analysis is to:

1.) "Deploy" the binomal tree model, by estimating options prices.

2.) Compare the "estimated" option prices with "real" option prices. (please compare these values by creating two scatter plots! One for calls, and another for puts) [example plot](https://cdn-images-1.medium.com/max/1000/1*7nl-TN6iO09mFxBLjQzubA.jpeg)

The dataset above is real option prices from YAHOO! Finance. 

Below is a sample table:

| Strike Price | Option Price ("Actual Value") | Time to Expiration (Years) | Model Estimate ("Predicted Value") |
| :---        |    :----    |          :--- | :--- |
|  1700      |  25.00     | 0.008  | $$P_{1}$$   |
| 1705   |  26.25         | 0.008  | $$P_{2}$$   |


The goal is to use the following as **constant** variables: 

- Stock Price ($S_{0} = 1213.859985$)

- Annual Volatility ($\sigma = 0.64946084322086 $)

- Dividend Rate ($d = 0$)

- Risk-Free Rate ($r = 0.00045$)





The goal is to solve for $P_{1}$ , $P_{2}$ , $P_{3}$ , ...

The other variables change for every row in our data! These are **not** constant


- Option Type (Call or Put)

- Strike Price ($K$)

- Time to Maturity ($T$)


#### As a final note, the options data is a snapshot in time. In other words, this was the current options prices for Nov. 3rd 2021

---

### Begin Model Deployment


In [35]:


s = 1213.859985        # Stock Price at Time = 0
v = .64946084322086    # Volatility
v = .62
r = .00045             # risk-free rate
d = 0                  # dividend yeild
nsteps = 50            # number of time steps .... Anything beyond 50 could be extreme.


k = 1200       # Strike Price
t = 0.007936508   # time to maturity (in years)



In [37]:
df = derivmkts::binomopt(s, k , v , r , t , d , nsteps,
                         putopt=FALSE,    # <----- putopt = FALSE (price for call option) 
                         american=TRUE,   # <----- american = TRUE (price for american option). This should always be TRUE
                         returntrees=TRUE)# <----- returntrees = TRUE (returns the option price). This should always be TRUE


In [38]:
df

34.40027,40.07246,46.31669,53.12297,60.47052,68.32857,76.657988,85.413331,94.545428,104.004079,⋯,472.64503619,485.76453806,498.9870828,512.3134792,525.744542,539.2810939,552.923962,566.673980,580.531991,594.49884
0.00000,28.77271,33.87738,39.56402,45.83335,52.67446,60.064938,67.971860,76.353506,85.161719,⋯,446.71833900,459.63434086,472.6517883,485.7714776,498.994211,512.3207980,525.752053,539.288798,552.931861,566.68208
0.00000,0.00000,23.70817,28.23547,33.34403,39.04613,45.342238,52.220393,59.656398,67.614915,⋯,421.19353498,433.90919133,446.7247207,459.6409071,472.658541,485.7784172,499.001340,512.328117,525.759564,539.29650
0.00000,0.00000,0.00000,19.21643,23.16707,27.68677,32.799564,38.518247,44.842988,51.760655,⋯,396.06439435,408.58281075,421.1995521,433.9153901,446.731103,459.6474735,472.665293,485.785357,499.008468,512.33544
0.00000,0.00000,0.00000,0.00000,15.29681,18.68286,22.614158,27.125844,32.243265,37.979786,⋯,371.32478386,383.64901777,396.0700524,408.5886477,421.205569,433.9215889,446.737484,459.654040,472.672045,485.79230
0.00000,0.00000,0.00000,0.00000,0.00000,11.93731,14.782424,18.137897,22.048628,26.551837,⋯,346.96866536,359.10172682,371.3300885,383.6544985,396.075711,408.5944847,421.211587,433.927788,446.743866,459.66061
0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,9.114507,11.453270,14.257853,17.580772,⋯,322.99009429,334.93494670,346.9736221,359.1068569,371.335393,383.6599793,396.081369,408.600322,421.217604,433.93399
0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,6.794071,8.670671,10.960992,⋯,299.38321823,311.14277904,322.9947085,334.9397315,346.978579,359.1119870,371.340698,383.665460,396.087027,408.60616
0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,4.932170,6.398295,⋯,276.14227550,287.71941693,299.3874952,311.1472240,322.999323,334.9445164,346.983536,359.117117,371.346003,383.67094
0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,3.477522,⋯,253.26159371,264.65914348,276.1462204,287.7235272,299.391772,311.1516690,323.003937,334.949301,346.988493,359.12225
0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,⋯,230.73558841,241.95633038,253.2652118,264.6629244,276.150165,287.7276376,299.396049,311.156114,323.008551,334.95409


In [42]:
print("This is the price estimate of the option using binomal tree")
as.numeric(df$price)

[1] "This is the price estimate of the option using binomal tree"


[1] 34.40027

### Congradulations ! Now, repeat the process for all call and put options in the data!